# Getting Start

### Installation of JanusQ
JanusQ can be installed by docker, wheel, and soruce code. Docker is recommended, as all functions have been tested on it.
Linux (Ubuntu 22.04 latest) platform and Python (3.10) is prefered when instaling by wheel or soruce code.

**From docker.**
Pull docker using docker 
        
        docker pull jxhhhh/janusq:latest

The code is in "/home/JanusQ-main". The examples that can be directly run is in "/home/JanusQ-main/examples"

**From wheel.**
Download janusq.whl from "JanusQ-main/dist".

        pip install janusq.whl

**From source code.**
Run following commends.

        git clone git@github.com:JanusQ/JanusQ.git
        cd JanusQ
        pip install -r requirements.txt

### Structure of JanusQ

- JanusCT
  - vectorization
    - janusq/analysis/vectorization.py: This python script holds code related to the generation of path table and the vecterization of circuits
  - fidelity preidiction
    - janusq/analysis/fidelity_prediction.py: This python script holds code related to training a model and using it to predict the fidelity of a circuit.
  - fidelity optimization
    -  janusq/optimizations/mapping/mapping_ct.py：This python script holds code related to a typical compilation flow includes routing and scheduling. The routing pass transforms the circuit to satisfy the processor topology.
  - unitary decomposition
    - janusq/analysis/unitary_decompostion.py: This python script holds code related to takes a unitary as input and decomposes it into matrices of basic gates, resulting in an equivalent circuit.
  - bug identification: identify the potential bugs in the quantum algorithm implementation.
- JanusFEM
  - benchmarking.py. Generate a circuit for measuring calibration matrices.
  - mitigation.py. Implement iterative calibration, which includes quantifying interactions between qubits and constructing Bayesian networks.
  - tools.py. Implement data format conversion.
- HyQSAT
  - hyqsat/common: This dir stores common functions about cnf files, such as readCNF.
  - hyqsat/solveSatBy**.py: The main python function; use to solve sat problem.
- time crystal

### Note
1. Permission denied 'minisat_core'  
This file have no executable permission. You need run `chmod +x janusq/hyqsat/minisat_core` in bash.

### Test JanusQ

In [1]:
import sys
sys.path.append('..')

import logging
logging.basicConfig(level=logging.ERROR)

#### Test Janus-CT

In [2]:
# test janus-ct
from janusq.data_objects.backend import LinearBackend
from janusq.data_objects.random_circuit import random_circuits
from janusq.analysis.vectorization import RandomwalkModel
# define a 10-qubit backend
n_qubits = 8
n_steps = 1
n_walks = 20
backend = LinearBackend(n_qubits, 1)

circuits = random_circuits(backend, n_circuits=300, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

vec_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
vec_model.train(circuits, multi_process=False, remove_redundancy = False)

INFO:root:start random walk for 300 circuits
100%|██████████| 300/300 [00:02<00:00, 136.92it/s]
INFO:root:count path
INFO:root:device size after random walk = 15
INFO:root:0's path table size = 11
INFO:root:1's path table size = 16
INFO:root:2's path table size = 19
INFO:root:3's path table size = 19
INFO:root:4's path table size = 19
INFO:root:5's path table size = 19
INFO:root:6's path table size = 16
INFO:root:7's path table size = 11
INFO:root:(0, 1)'s path table size = 13
INFO:root:(1, 2)'s path table size = 18
INFO:root:(3, 4)'s path table size = 21
INFO:root:(2, 3)'s path table size = 21
INFO:root:(6, 7)'s path table size = 13
INFO:root:(4, 5)'s path table size = 21
INFO:root:(5, 6)'s path table size = 18


#### Test Janus-FEM

In [3]:
# test readout calibration
from janusq.simulator.noisy_simulator import NoisySimulator
from janusq.data_objects.backend import LinearBackend
from janusq.simulator.readout_error_model import ReadoutErrorModel
from janusq.optimizations.readout_mitigation.fem import  EnumeratedProtocol
from janusq.data_objects.algorithms import get_algorithm_circuits

n_qubits = 3
backend = LinearBackend(n_qubits, 1)
circuit = get_algorithm_circuits(n_qubits, backend, algs = ['ghz'])[0]

samples = 10000

simulator = NoisySimulator(
    backend, readout_error_model = ReadoutErrorModel.random_model(backend))
output_noise = simulator.execute(circuit,samples)


protocol = EnumeratedProtocol(n_qubits)
real_bstrs, circuits_protocol = protocol.gen_circuits()

all_statuscnts = [
    simulator.execute(cir_protocol, samples)
    for cir_protocol in circuits_protocol
]

print(all_statuscnts)


INFO:qiskit.passmanager.base_tasks:Pass: UnrollCustomDefinitions - 0.15783 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('u3', 1)} to target basis {'z', 'ry', 'snapshot', 'cx', 'rz', 'barrier', 'measure', 'u', 'reset', 'rx', 'x', 'p', 'delay'}.
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Begining basis search from {('u3', 1)} to {'z', 'ry', 'snapshot', 'cx', 'rz', 'barrier', 'measure', 'u', 'reset', 'rx', 'x', 'p', 'delay'}.
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Gate u1 generated using rule 
global phase: theta/2
   ┌───────────┐
q: ┤ Rz(theta) ├
   └───────────┘
 with total cost of 1.0.
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Gate id generated using rule 
   ┌───────┐
q: ┤ Rz(0) ├
   └───────┘
 with total cost of 1.0.
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Gate tdg generated using rule 
   ┌──────────┐
q: ┤ U1(-π/4) ├
   └──────────┘
 with total cost of 1.0.
DEBUG:qiskit.transp

[{'111': 7, '011': 43, '101': 58, '110': 61, '100': 712, '001': 582, '000': 7899, '010': 638}, {'100': 5, '010': 1, '000': 26, '011': 704, '111': 65, '101': 790, '001': 8409}, {'110': 50, '111': 4, '011': 50, '001': 557, '101': 40, '100': 696, '010': 662, '000': 7941}, {'101': 5, '100': 54, '001': 52, '111': 53, '011': 577, '110': 731, '000': 562, '010': 7966}, {'110': 1, '101': 45, '001': 609, '000': 1, '010': 19, '111': 762, '011': 8563}, {'101': 6, '001': 36, '111': 57, '011': 590, '100': 56, '110': 723, '000': 551, '010': 7981}, {'110': 64, '111': 6, '011': 43, '101': 45, '001': 575, '100': 709, '010': 681, '000': 7877}, {'100': 2, '011': 695, '111': 58, '010': 5, '000': 26, '101': 761, '001': 8453}, {'101': 53, '111': 6, '011': 49, '001': 518, '100': 746, '110': 61, '010': 685, '000': 7882}, {'001': 69, '011': 6, '111': 39, '101': 575, '010': 62, '000': 813, '110': 704, '100': 7732}, {'110': 2, '000': 1, '100': 24, '011': 74, '111': 737, '001': 875, '101': 8287}, {'001': 79, '101'

#### Test Janus-SAT

In [4]:
# test hyqsat calibration
# input cnf flie
file_path = "cnf_examples/uf50-01.cnf"
from janusq.hyqsat import readCNF

print(readCNF(file_path))

([(-3, 7, 36), (-3, -42, -48), (-41, -47, -49), (8, 17, -40), (-21, -31, -39), (-22, 36, 49), (14, 27, 38), (6, 15, -18), (6, 7, -43), (-7, 23, 34), (2, -13, 14), (2, -42, 47), (3, -33, -35), (40, 44, 49), (31, 36, 50), (-3, -36, -37), (26, -29, 43), (15, 29, -45), (-11, 18, 24), (6, -26, -47), (-10, -33, -50), (6, 16, 32), (-34, 37, 41), (7, -17, -28), (19, -44, 46), (7, 22, -48), (3, 34, 39), (31, -43, 46), (23, -27, 32), (-18, 37, -50), (5, 11, 20), (6, -24, -45), (-14, -23, -34), (20, 21, -22), (-17, 24, 50), (-24, -25, -27), (3, 21, 35), (-26, -36, 47), (-28, -45, 49), (-6, 12, -21), (-15, -17, -39), (2, -14, 41), (-23, 25, 36), (-3, -39, -40), (20, 35, 50), (27, 31, -39), (-15, -40, 45), (34, 35, 50), (-1, 12, -48), (18, -30, -35), (-24, -25, 27), (-4, -12, -33), (-24, -37, -43), (31, -37, -44), (-9, 14, -38), (-16, 33, 34), (4, -5, -35), (-3, -19, -21), (-29, -35, -36), (7, 36, -43), (14, 30, 41), (-7, -24, -35), (6, 35, -42), (-1, -15, 39), (-16, 27, 49), (-10, -37, 49), (-3, -